In [ ]:
import numpy as np

In [ ]:
class Vasicek:
    
    def __init__(self,kappa,theta,sigma): 
        self.kappa=kappa
        self.theta=theta
        self.sigma=sigma

In [ ]:
hw3dynamics = Vasicek(kappa=3,theta=0.05,sigma=0.03)

In [ ]:
class Bond:
    
    def __init__(self, T):
        self.T=T
    

In [ ]:
hw3contract = Bond(T=5)

In [ ]:
class FDexplicit:
    
    def __init__(self, rMax, rMin, deltar, deltat, useUpwind):         
        self.rMax=rMax
        self.rMin=rMin
        self.deltar=deltar
        self.deltat=deltat
        self.useUpwind=useUpwind
        
    
    def price_bond_vasicek(self,contract,dynamics):
    # You complete the coding of this function
    #
    # Returns array of all initial short rates,
    # and the corresponding array of zero-coupon
    # T-maturity bond prices

        T = contract.T
        N=round(T/self.deltat)
        if abs(N-T/self.deltat) > 1e-12:
            raise ValueError("Bad delta t")

        r=np.arange(self.rMax,self.rMin-self.deltar/2,-self.deltar)   #I'm making the FIRST indices of the array correspond to HIGH levels of r
        bondprice=np.ones(np.size(r))

        if self.useUpwind:
            qu=    #fill this in with an array.
            qd=    #fill this in with an array.
            qm=    #fill this in with an array.
        else:
            qu=    #fill this in with an array.
            qd=    #fill this in with an array.
            qm=    #fill this in with an array.

        for t in np.arange(N-1,-1,-1)*self.deltat:
            bondprice=1/(1+r*self.deltat)*(qd*np.roll(bondprice,-1)+qm*bondprice+qu*np.roll(bondprice,1))

            # It is not obvious in this case, 
            # what boundary conditions to use at the top and bottom
            # so let us assume "linearity" boundary conditions
            bondprice[0]=2*bondprice[1]-bondprice[2]
            bondprice[-1]=2*bondprice[-2]-bondprice[-3]

        return (r, bondprice)

In [ ]:
hw3FD = FDexplicit(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=False)

In [ ]:
(r, bondprice) = hw3FD.price_bond_vasicek(hw3contract,hw3dynamics)

In [ ]:
np.set_printoptions(precision=4,suppress=True)
displayrows=(r<0.15+hw3FD.deltar/2) & (r>0.0-hw3FD.deltar/2)

In [ ]:
print(np.stack((r, bondprice),1)[displayrows])